![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# import

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
from datetime import datetime, timedelta

import re

In [2]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [3]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [4]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [5]:
df_train.info()

In [6]:
df_train.head(5)

In [7]:
df_test.info()

In [8]:
df_test.head(5)

In [9]:
sample_submission.head(5)

In [10]:
sample_submission.info()

In [11]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [12]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [13]:
for i in data.columns:
    print(i + ' - ' + str(type(data[i][0])))

In [14]:
data.sample(5)

In [15]:
data.Reviews[1]

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

Сменим названия столбцов

In [16]:
#data.columns = ['restaurant_id', 'city', 'cuisine', 'ranking', 'price', 
#                'number_of_reviews', 'reviews', 'url', 'id_ta', 'sample', 'rating']

# functions

In [17]:
def price_rating(x):
    if x == '$':
        return 1
    elif x == '$$ - $$$':
        return 2
    elif x == '$$$$':
        return 3
    else:
        return 0

In [18]:

def cuisine_clean(x):
    x = str(x).replace('[', '')
    x = x.replace(']', '')
    x = x.replace('\\', '')
    x = x.replace("'", '')
    x = x.replace(" ", '')
    x = x.split(',')
    return x

In [19]:
def split_date_fast(row):
    x = str(row).split('],')
    if len(x) == 2:
        x = x[1].replace(']','').replace('[','').replace("'",'').replace(" ",'')
        x = x.split(',')
        if (len(x) == 2)and(len(x[1]) >= 10):
            return datetime.strptime(x[1], '%m/%d/%Y').timestamp()
        elif len(x[0]) >= 10:
            return datetime.strptime(x[0], '%m/%d/%Y').timestamp()
        else:
            return None
    else:
        return None

In [20]:
def split_date_last(row):
    x = str(row).split('],')
    if len(x) == 2:
        x = x[1].replace(']','').replace('[','').replace("'",'').replace(" ",'')
        x = x.split(',')
        if (len(x) == 2)and(len(x[0]) >= 10):
            return datetime.strptime(x[0], '%m/%d/%Y').timestamp()
        else:
            return None
    else:
        return None

In [21]:
def ranking_norm(row):
    global city_dict
    return row.Ranking / city_dict[row.City]

In [22]:
def cuisine_other(row):
    global list_out
    tmp_list = set()
    for value in row:
        if value in list_out:
            tmp_list.add('Other')
        else:
            tmp_list.add(value)

    return tmp_list
            

In [23]:
def cuisine_popular(row):
    global list_popular
    for value in row['Cuisine Style']:
        if not (value in list_popular):
            row['Other_popular'] == 1
        else:
            if list_popular.index(value) == 1:
                row['1_popular'] == 1
                row['2_popular'] == 0
                row['3_popular'] == 0
            elif list_popular.index(value) == 2:
                row['1_popular'] == 0
                row['2_popular'] == 1
                row['3_popular'] == 0
            elif list_popular.index(value) == 3:
                row['1_popular'] == 0
                row['2_popular'] == 0
                row['3_popular'] == 1
        

    return row

In [24]:
def cuisine_popular_1(row):
    global list_popular
    for value in row:
        if value in list_popular:
            if list_popular.index(value) == 0:
                return 1
            else:
                return 0
        else:
            return 0

In [25]:
def cuisine_popular_2(row):
    global list_popular
    for value in row:
        if value in list_popular:
            if list_popular.index(value) == 1:
                return 1
            else:
                return 0
        else:
            return 0

In [26]:
def cuisine_popular_3(row):
    global list_popular
    for value in row:
        if value in list_popular:
            if list_popular.index(value) == 2:
                return 1
            else:
                return 0
        else:
            return 0

In [27]:
def cuisine_popular_other(row):
    global list_popular
    for value in row:
        if not (value in list_popular):
            return 1
        else:
            return 0

In [28]:
def pub(row):
    for value in row:
        if value in ['Bar', 'Pub', 'BrewPub','WineBar', 'Grill', 'Gastropub']:
            return 1
        else:
            return 0

In [29]:
def vegan(row):
    for value in row:
        if value in ['VegetarianFriendly', 'VeganOptions', 'GlutenFreeOptions', 'Healthy']:
            return 1
        else:
            return 0

In [30]:
def cafe(row):
    for value in row:
        if value in ['Cafe', 'FastFood', 'Pizza',  'Steakhouse', 'Soups', 'Sushi', 'Diner']:
            return 1
        else:
            return 0

In [31]:
def national_cuisine(row):
    for value in row:
        if value in ['Scandinavian', 'French', 'Portuguese', 'Greek', 'Czech',  'British', 'Indian', 
                     'Thai', 'Dutch', 'Welsh', 'American', 'Hungarian', 'Spanish', 'Norwegian', 'Moroccan', 
                     'Scottish', 'Peruvian', 'Irish', 'Polish', 'Vietnamese', 'Lebanese', 'Turkish', 
                     'Japanese', 'Austrian', 'Chinese', 'Danish', 'Korean', 'German',  'Arabic',  
                     'Swiss', 'Swedish', 'Persian', 'Australian',  'Mexican','Belgian', 'Argentinean', 
                     'Tunisian', 'Russian', 'Nepali', 'Bangladeshi', 'Colombian', 'SriLankan', 'Pakistani', 
                     'Israeli', 'Brazilian', 'Delicatessen',  'Ethiopian', 'Taiwanese', 'Indonesian', 'Malaysian',
                     'Venezuelan', 'Balti', 'Armenian', 'Slovenian', 'Caribbean', 'Jamaican', 'Ecuadorean',
                     'Chilean', 'Hawaiian', 'MinorityChinese', 'Croatian', 'Cambodian', 'Afghani', 'Tibetan', 
                     'Romanian', 'Mongolian', 'Cajun&Creole', 'Singaporean', 'Filipino', 'Cuban', 'NativeAmerican', 
                     'Canadian', 'Egyptian', 'Azerbaijani', 'Georgian', 'Yunnan', 'NewZealand', 'Ukrainian', 
                     'Albanian','Polynesian', 'Caucasian', 'Uzbek', 'Salvadoran','Xinjiang', 'Burmese', 
                     'Fujian', 'Latvian']:
            return 1
        else:
            return 0

In [32]:
def territorial_cuisine(row):
    for value in row:
        if value in ['European', 'Mediterranean', 'EasternEuropean', 'CentralEuropean',  'MiddleEastern',
                     'African', 'Asian', 'CentralAmerican', 'SouthAmerican', 'International', 'Latin', 
                     'Southwestern', 'CentralAsian']:
            return 1
        else:
            return 0

In [33]:
def religious_cuisine(row):
    for value in row:
        if value in ['Halal', 'Kosher']:
            return 1
        else:
            return 0

In [34]:
def clean_id_res(row):
    return int(str(row[3:]))

In [35]:
def clean_id_ta(row):
    return int(str(row[1:]))

In [36]:
data.Reviews[1]

In [37]:
def positive_feedback(row):
    list_words = ('well', 'good','best', 'friendly', 'nice', 'ok', 'great', 'lovely','excellent', 'beautiful',
                 'Well', 'Good','Best', 'Friendly', 'Nice', 'Ok', 'Great', 'Lovely','Excellent', 'Beautiful')
    feedback = str(row).split("], [")
    feedback = feedback[0].replace(']',' ').replace('[',' ').replace('!',' ').replace('?',' ')\
                          .replace('.',' ').replace("'",' ').split()
    for words in list_words:
        if words in feedback:
            return 1
        else:
            return 0


In [38]:
def negative_feedback(row):
    list_words = ('poor', 'bad', 'worse', 'Poor', 'Bad', 'Worse')
    feedback = str(row).split("], [")
    feedback = feedback[0].replace(']',' ').replace('[',' ').replace('!',' ').replace('?',' ')\
                          .replace('.',' ').replace("'",' ').split()
    for words in list_words:
        if words in feedback:
            return 1
        else:
            return 0

In [39]:
def get_boxplot(df: pd.DataFrame, column: object, ax_y='Rating'):
    fig, ax = plt.subplots(figsize = (10, 10))
    box_plot = sns.boxplot(x=column, y=ax_y, data=df,ax=ax)
    cnt = df[column].value_counts()
    medians = data.groupby([column])[ax_y].median()
    vertical_offset = data.groupby([column])[ax_y].median() * 0.12
    x=data[column].unique()
    x = list(filter(lambda v: v==v, x))
    if df.loc[:, col].dtypes != np.dtype('O'):
        x = sorted(x)
    for xtick, ytick in zip(list(box_plot.get_xticks()), x):
        box_plot.text(xtick, medians.loc[ytick]-vertical_offset[ytick], cnt.loc[ytick], 
                    horizontalalignment='center',size=15, color='w',weight='semibold')
    plt.xticks(rotation=45)
    ax.set_title('Boxplot for ' + column)
    plt.show()

In [40]:
def get_FacetGrid(data: pd.DataFrame, column: list, id_vars='Rating'):
    column.append(id_vars)
    sns.FacetGrid(data[column].melt(id_vars=[id_vars], var_name="feature",value_name="value"), 
                  col="feature").map(sns.boxplot, "value", "Rating")

# Cleaning and Prepping Data
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   
![](https://analyticsindiamag.com/wp-content/uploads/2018/01/data-cleaning.png)

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

### 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [41]:
data.nunique(dropna=False)

Какие признаки можно считать категориальными?

Для кодирования категориальных признаков есть множество подходов:
* Label Encoding
* One-Hot Encoding
* Target Encoding
* Hashing

Выбор кодирования зависит от признака и выбраной модели.
Не будем сейчас сильно погружаться в эту тематику, давайте посмотрим лучше пример с One-Hot Encoding:
![](https://i.imgur.com/mtimFxh.png)

In [42]:
city_dict = dict(data.City.value_counts())
data['ranking_norm'] = data.apply(ranking_norm, axis=1)
data

In [44]:
fig, ax = plt.subplots(figsize = (15, 8))
box_plot = sns.boxplot(x='City', y='Rating', data=data, ax=ax)
cnt = data['City'].value_counts()
medians = data.groupby(['City'])['Rating'].median()
vertical_offset = data.groupby(['City'])['Rating'].median() * 0.12
x=data['City'].unique()
x = list(filter(lambda v: v==v, x))

    
plt.xticks(rotation=45)
ax.set_title('Boxplot for City')
plt.show()

In [45]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
data = pd.get_dummies(data, columns=['City'], dummy_na=True)

In [46]:
data.head(5)

In [47]:
data.sample(5)

#### Возьмем следующий признак "Price Range".

In [48]:
data['Price Range'].value_counts()

По описанию 'Price Range' это - Цены в ресторане.  
Их можно поставить по возрастанию (значит это не категориальный признак). А это значит, что их можно заменить последовательными числами, например 1,2,3  
*Попробуйте сделать обработку этого признака уже самостоятельно!*

In [49]:
# Ваша обработка 'Price Range'
data['Price Range'] = data['Price Range'].apply(price_rating)

> Для некоторых алгоритмов МЛ даже для не категориальных признаков можно применить One-Hot Encoding, и это может улучшить качество модели. Пробуйте разные подходы к кодированию признака - никто не знает заранее, что может взлететь.

### Обработать другие признаки вы должны самостоятельно!
Для обработки других признаков вам возможно придется даже написать свою функцию, а может даже и не одну, но в этом и есть ваша практика в этом модуле!     
Следуя подсказкам в модуле вы сможете более подробно узнать, как сделать эти приобразования.

In [50]:
data['ID_TA'].explode().nunique()

In [51]:
data['Restaurant_id'].explode().nunique()

In [52]:
data['Restaurant_id'].explode().value_counts()

In [53]:
data['ID_TA'].value_counts()

In [54]:
data[data['ID_TA'] == 'd11999956']

In [55]:
data[data['Restaurant_id'] == 'id_871']

имеются дубликаты ресторанов имеющие разные данные в остальных столбцах dataframe. 

In [56]:
data['Restaurant_id'] = data['Restaurant_id'].apply(clean_id_res)
data['ID_TA'] = data['ID_TA'].apply(clean_id_ta)
data[['Restaurant_id', 'ID_TA']].corr()


Повторение ресторанов может обьяснятся тем что это сеть

In [57]:
restaurant_list = list(data['Restaurant_id'].value_counts()[data['Restaurant_id'].value_counts() > 1].index)
data['network_res'] = data[data['Restaurant_id'].isin(restaurant_list)]['Restaurant_id'].apply(lambda x: 1)
data['network_res'].fillna(0, inplace=True)
data['network_res'].value_counts()

In [58]:
data['Cuisine_NAN'] = pd.isnull(data['Cuisine Style']).astype('uint8')

In [59]:
data['Cuisine Style'] = data['Cuisine Style'].apply(cuisine_clean)


используем все виды Cuisine Style разбив на категории

In [60]:
data['Cuisine Style'].explode().unique()

In [61]:
data['vegan'] = data['Cuisine Style'].apply(vegan)
data['pub'] = data['Cuisine Style'].apply(pub)
data['cafe'] = data['Cuisine Style'].apply(cafe)

In [62]:
data['cuisine_sum'] = data['Cuisine Style'].apply(lambda x: len(x))

In [63]:
data['Cuisine Style'].explode().value_counts()

Выделим тройку самых популярных Cuisine Style

In [64]:
#list_out = data['Cuisine Style'].explode().value_counts()
#list_out = list(list_out[list_out < 100].index)
list_popular = list(data['Cuisine Style'].explode().value_counts(ascending=False).drop(index='nan')[:3].keys())
list_popular


In [65]:
#data['Cuisine Style'] = data['Cuisine Style'].apply(cuisine_other)
#data = data.apply(cuisine_popular, axis=1)
data['1_popular'] = data['Cuisine Style'].apply(cuisine_popular_1)
data['2_popular'] = data['Cuisine Style'].apply(cuisine_popular_2)
data['3_popular'] = data['Cuisine Style'].apply(cuisine_popular_3)
data['religious_cuisine'] = data['Cuisine Style'].apply(religious_cuisine)
data['national_cuisine'] = data['Cuisine Style'].apply(national_cuisine)
data['territorial_cuisine'] = data['Cuisine Style'].apply(territorial_cuisine)

data['Other_popular'] = data['Cuisine Style'].apply(cuisine_popular_other)
#data = pd.get_dummies(data, columns=['Cuisine Style'], dummy_na=True)
data.info()

In [66]:
data

In [67]:
data['positive_feedback'] = data.Reviews.apply(positive_feedback)
data['negative_feedback'] = data.Reviews.apply(negative_feedback)
data['date_fast'] = data.Reviews.apply(split_date_fast)
data['date_last'] = data.Reviews.apply(split_date_last)

data['date_fast_NAN'] = pd.isnull(data['date_fast']).astype('uint8')
data['date_last_NAN'] = pd.isnull(data['date_last']).astype('uint8')

data.date_last.fillna(value = 0, inplace=True)
data.date_fast.fillna(value = 0, inplace=True)
data['date_diff'] = abs(data['date_last'] - data['date_fast'])

In [68]:
data.positive_feedback.nunique()

In [69]:
data['Number of Reviews'].fillna(value = 0, inplace=True)

![](https://cs10.pikabu.ru/post_img/2018/09/06/11/1536261023140110012.jpg)

# EDA 
[Exploratory Data Analysis](https://ru.wikipedia.org/wiki/Разведочный_анализ_данных) - Анализ данных
На этом этапе мы строим графики, ищем закономерности, аномалии, выбросы или связи между признаками.
В общем цель этого этапа понять, что эти данные могут нам дать и как признаки могут быть взаимосвязаны между собой.
Понимание изначальных признаков позволит сгенерировать новые, более сильные и, тем самым, сделать нашу модель лучше.
![](https://miro.medium.com/max/2598/1*RXdMb7Uk6mGqWqPguHULaQ.png)

### Посмотрим распределение признака

In [70]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [71]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [72]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [73]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

>Подумайте как из этого можно сделать признак для вашей модели. Я покажу вам пример, как визуализация помогает находить взаимосвязи. А далее действуйте без подсказок =) 


### Посмотрим распределение целевой переменной

In [74]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [75]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [76]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### И один из моих любимых - [корреляция признаков](https://ru.wikipedia.org/wiki/Корреляция)
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [77]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(),)

Вообще благодаря визуализации в этом датасете можно узнать много интересных фактов, например:
* где больше Пицерий в Мадриде или Лондоне?
* в каком городе кухня ресторанов более разнообразна?

придумайте свои вопрос и найдите на него ответ в данных)

In [78]:
for i in data.columns:
    print(i + ' - ' + str(type(data[i][0])))

In [79]:
fig, ax = plt.subplots(figsize = (15, 5))
sns.boxplot(x='Price Range', y='Rating',data=data.loc[data.loc[:, 'Price Range'].isin(data.loc[:, 'Price Range'].value_counts().index[:])],ax=ax)
plt.xticks(rotation=90)
ax.set_title(f'Boxplot for Price Range')
plt.show()

Price Range не оказывает существенного влияния на Rating

In [80]:
columns = ['1_popular', '2_popular', '3_popular', 'Other_popular']
get_FacetGrid(data, columns)

#sns.FacetGrid(data[['1_popular', '2_popular', '3_popular', 'Other_popular', 'Rating']].melt(id_vars=['Rating'],
#                         var_name="feature",value_name="value"), col="feature").map(sns.boxplot, "value", "Rating")

Выделение популярных Cuisine Style не дает особого эфекта

In [81]:
columns = ['positive_feedback', 'negative_feedback']
get_FacetGrid(data, columns)

Как видно неготивные отзывы снижают Rating

In [82]:
columns = ['Cuisine_NAN', 'vegan', 'pub', 'cafe', 'religious_cuisine', 'national_cuisine', 'territorial_cuisine']
get_FacetGrid(data, columns)

Разделение по Cuisine Style на группы особого эффекта не дает 

In [83]:
columns = ['network_res']
get_FacetGrid(data, columns)

Как видно сетевые рестораны имеют больший Rating

In [84]:
columns = ['date_fast_NAN', 'date_last_NAN']
get_FacetGrid(data, columns)

In [85]:
data[['date_fast_NAN', 'date_last_NAN']].corr()

Данные признаки имеют большую корреляциию. Целесообразно использовать один из признаков

In [86]:
for i in data.columns:
    print(i + ' - ' + str(type(data[i][0])))

In [87]:
corr = data[['Rating', 'Restaurant_id', 'Ranking', 'Number of Reviews', 'ID_TA', 'ranking_norm', 
             'cuisine_sum', 'date_fast', 'date_last', 'date_diff']].corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(12, 12))
    ax = sns.heatmap(corr, mask=mask, annot=True, square=True)

Видна значительная корреляция между cuisine_sum и ranking_norm, ranking_norm и Number of Reviews, date_fast и date_last, date_last и с date_diff

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [88]:
# на всякий случай, заново подгружаем данные
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [89]:
def price_rating(x):
    if x == '$':
        return 1
    elif x == '$$ - $$$':
        return 2
    elif x == '$$$$':
        return 3
    else:
        return 0
    

def cuisine_clean(x):
    x = str(x).replace('[', '')
    x = x.replace(']', '')
    x = x.replace('\\', '')
    x = x.replace("'", '')
    x = x.replace(" ", '')
    x = x.split(',')
    return x


def split_date_fast(row):
    x = str(row).split('],')
    if len(x) == 2:
        x = x[1].replace(']','').replace('[','').replace("'",'').replace(" ",'')
        x = x.split(',')
        if (len(x) == 2)and(len(x[1]) >= 10):
            return datetime.strptime(x[1], '%m/%d/%Y').timestamp()
        elif len(x[0]) >= 10:
            return datetime.strptime(x[0], '%m/%d/%Y').timestamp()
        else:
            return None
    else:
        return None
    

def split_date_last(row):
    x = str(row).split('],')
    if len(x) == 2:
        x = x[1].replace(']','').replace('[','').replace("'",'').replace(" ",'')
        x = x.split(',')
        if (len(x) == 2)and(len(x[0]) >= 10):
            return datetime.strptime(x[0], '%m/%d/%Y').timestamp()
        else:
            return None
    else:
        return None
    
    
def ranking_norm(row):
    global city_dict
    return row.Ranking / city_dict[row.City]


def cuisine_other(row):
    global list_out
    tmp_list = set()
    for value in row:
        if value in list_out:
            tmp_list.add('Other')
        else:
            tmp_list.add(value)
    return tmp_list


def cuisine_popular_1(row):
    global list_popular
    for value in row:
        if value in list_popular:
            if list_popular.index(value) == 0:
                return 1
            else:
                return 0
        else:
            return 0
        

def cuisine_popular_2(row):
    global list_popular
    for value in row:
        if value in list_popular:
            if list_popular.index(value) == 1:
                return 1
            else:
                return 0
        else:
            return 0
        
        
def cuisine_popular_3(row):
    global list_popular
    for value in row:
        if value in list_popular:
            if list_popular.index(value) == 2:
                return 1
            else:
                return 0
        else:
            return 0
        
def cuisine_popular_other(row):
    global list_popular
    for value in row:
        if not (value in list_popular):
            return 1
        else:
            return 0
        
        
def pub(row):
    for value in row:
        if value in ['Bar', 'Pub', 'BrewPub','WineBar', 'Grill', 'Gastropub']:
            return 1
        else:
            return 0
        
        
def vegan(row):
    for value in row:
        if value in ['VegetarianFriendly', 'VeganOptions', 'GlutenFreeOptions', 'Healthy']:
            return 1
        else:
            return 0
        
def cafe(row):
    for value in row:
        if value in ['Cafe', 'FastFood', 'Pizza',  'Steakhouse', 'Soups', 'Sushi', 'Diner']:
            return 1
        else:
            return 0
        

def national_cuisine(row):
    for value in row:
        if value in ['Scandinavian', 'French', 'Portuguese', 'Greek', 'Czech',  'British', 'Indian', 
                     'Thai', 'Dutch', 'Welsh', 'American', 'Hungarian', 'Spanish', 'Norwegian', 'Moroccan', 
                     'Scottish', 'Peruvian', 'Irish', 'Polish', 'Vietnamese', 'Lebanese', 'Turkish', 
                     'Japanese', 'Austrian', 'Chinese', 'Danish', 'Korean', 'German',  'Arabic',  
                     'Swiss', 'Swedish', 'Persian', 'Australian',  'Mexican','Belgian', 'Argentinean', 
                     'Tunisian', 'Russian', 'Nepali', 'Bangladeshi', 'Colombian', 'SriLankan', 'Pakistani', 
                     'Israeli', 'Brazilian', 'Delicatessen',  'Ethiopian', 'Taiwanese', 'Indonesian', 'Malaysian',
                     'Venezuelan', 'Balti', 'Armenian', 'Slovenian', 'Caribbean', 'Jamaican', 'Ecuadorean',
                     'Chilean', 'Hawaiian', 'MinorityChinese', 'Croatian', 'Cambodian', 'Afghani', 'Tibetan', 
                     'Romanian', 'Mongolian', 'Cajun&Creole', 'Singaporean', 'Filipino', 'Cuban', 'NativeAmerican', 
                     'Canadian', 'Egyptian', 'Azerbaijani', 'Georgian', 'Yunnan', 'NewZealand', 'Ukrainian', 
                     'Albanian','Polynesian', 'Caucasian', 'Uzbek', 'Salvadoran','Xinjiang', 'Burmese', 
                     'Fujian', 'Latvian']:
            return 1
        else:
            return 0

        
def territorial_cuisine(row):
    for value in row:
        if value in ['European', 'Mediterranean', 'EasternEuropean', 'CentralEuropean',  'MiddleEastern',
                     'African', 'Asian', 'CentralAmerican', 'SouthAmerican', 'International', 'Latin', 
                     'Southwestern', 'CentralAsian']:
            return 1
        else:
            return 0
        
        
def religious_cuisine(row):
    for value in row:
        if value in ['Halal', 'Kosher']:
            return 1
        else:
            return 0


def clean_id_res(row):
    return int(str(row[3:]))




def clean_id_ta(row):
    return int(str(row[1:]))


def positive_feedback(row):
    list_words = ('well', 'good','best', 'friendly', 'nice', 'ok', 'great', 'lovely','excellent', 'beautiful',
                 'Well', 'Good','Best', 'Friendly', 'Nice', 'Ok', 'Great', 'Lovely','Excellent', 'Beautiful')
    feedback = str(row).split("], [")
    feedback = feedback[0].replace(']',' ').replace('[',' ').replace('!',' ').replace('?',' ')\
                          .replace('.',' ').replace("'",' ').split()
    for words in list_words:
        if words in feedback:
            return 1
        else:
            return 0

        
def negative_feedback(row):
    list_words = ('poor', 'bad', 'worse', 'Poor', 'Bad', 'Worse')
    feedback = str(row).split("], [")
    feedback = feedback[0].replace(']',' ').replace('[',' ').replace('!',' ').replace('?',' ')\
                          .replace('.',' ').replace("'",' ').split()
    for words in list_words:
        if words in feedback:
            return 1
        else:
            return 0


def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    data = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    #data.drop(['ID_TA'], axis = 1, inplace=True)
    
    
    # ################### 2. NAN ############################################################## 
    # Далее заполняем пропуски, вы можете попробовать заполнением средним или средним по городу и тд...
    data['Number of Reviews'].fillna(0, inplace=True)
    # тут ваш код по обработке NAN
    # ....
    
    
    # ################### 3. Encoding ############################################################## 
    # для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
    city_dict = dict(data.City.value_counts())
    data['Price Range'] = data['Price Range'].apply(price_rating)
    data['ranking_norm'] = data.apply(ranking_norm, axis=1)
    data = pd.get_dummies(data, columns=['City'], dummy_na=True)
    
    # тут ваш код не Encoding фитчей
    # ....
    
    
    # ################### 4. Feature Engineering ####################################################
    # тут ваш код не генерацию новых фитчей
    
    
    restaurant_list = list(data['Restaurant_id'].value_counts()[data['Restaurant_id'].value_counts() > 1].index)
    data['network_res'] = data[data['Restaurant_id'].isin(restaurant_list)]['Restaurant_id'].apply(lambda x: 1)
    data['network_res'].fillna(0, inplace=True)
    
    data['Restaurant_id'] = data['Restaurant_id'].apply(clean_id_res)
    data['ID_TA'] = data['ID_TA'].apply(clean_id_ta)
    
    data['Cuisine_NAN'] = pd.isnull(data['Cuisine Style']).astype('uint8')
    data['Cuisine Style'] = data['Cuisine Style'].apply(cuisine_clean)
    data['vegan'] = data['Cuisine Style'].apply(vegan)
    data['pub'] = data['Cuisine Style'].apply(pub)
    data['cafe'] = data['Cuisine Style'].apply(cafe)
    data['cuisine_sum'] = data['Cuisine Style'].apply(lambda x: len(x))
    list_popular = list(data['Cuisine Style'].explode().value_counts(ascending=False).drop(index='nan')[:3].keys())
    data['1_popular'] = data['Cuisine Style'].apply(cuisine_popular_1)
    data['2_popular'] = data['Cuisine Style'].apply(cuisine_popular_2)
    data['3_popular'] = data['Cuisine Style'].apply(cuisine_popular_3)
    data['religious_cuisine'] = data['Cuisine Style'].apply(religious_cuisine)
    data['national_cuisine'] = data['Cuisine Style'].apply(national_cuisine)
    data['territorial_cuisine'] = data['Cuisine Style'].apply(territorial_cuisine)
    
    data['positive_feedback'] = data.Reviews.apply(positive_feedback)
    data['negative_feedback'] = data.Reviews.apply(negative_feedback)
    data['date_fast'] = data.Reviews.apply(split_date_fast)
    data['date_last'] = data.Reviews.apply(split_date_last)
    #data['date_fast_NAN'] = pd.isnull(data['date_fast']).astype('uint8')
    data['date_last_NAN'] = pd.isnull(data['date_last']).astype('uint8')
    data.date_last.fillna(value = 0, inplace=True)
    data.date_fast.fillna(value = 0, inplace=True)
    #data['date_diff'] = abs(data['date_last'] - data['date_fast'])




    
    
    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in data.columns if data[s].dtypes == 'object']
    data.drop(object_columns, axis = 1, inplace=True)
    
    return data

>По хорошему, можно было бы перевести эту большую функцию в класс и разбить на подфункции (согласно ООП). 

#### Запускаем и проверяем что получилось

In [90]:
data = preproc_data(data)
data.sample(10)

In [91]:
data.info()

In [92]:
#data.drop([ ], axis = 1, inplace=True)

In [93]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [94]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [95]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [96]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [97]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [98]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [99]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [100]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [101]:
test_data.sample(10)

In [102]:
test_data = test_data.drop(['Rating'], axis=1)

In [103]:
sample_submission

In [104]:
predict_submission = model.predict(test_data)

In [105]:
predict_submission

In [106]:
sample_submission['rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!


## На данный момент можно найти еще некоторые признаки, что возможно приведет к снижению МАЕ, но наврятли ниже 0,18. Для действительно значительного улучшения модели необходимы еще данные (например: наиболее посещаемые города туристами, наличие аэропортов, туристический поток по странам и т.д.) на что просто не хватило времени